In [ ]:
from webpage.appcreator.import_utils import fetch_models
from archeutils.utils import ARCHE_BASE_URI, acdh_ns

In [ ]:
from django.utils.text import slugify

In [ ]:
from archeutils.utils import get_p4d_id

In [ ]:
def arche_actor_id(legacy_id):
    slug = slugify(legacy_id)
    return "/".join([ARCHE_BASE_URI, 'agents', slug])

In [ ]:
models = fetch_models('archiv')

In [ ]:
mapping = {
    'creator_metadata': 'hasMetadataCreator',
    'creator_original': 'hasCreator',
    'creator_archivalobject': 'hasContributor',
    'creator_scan': 'hasContributor',
}

In [ ]:
arche_dict = {}
for x in models:
    for key, value in mapping.items():
        filter_expr = {f"{key}__isnull": True}
        if hasattr(x, key):
#             print(f"{x} mapps {key} to {value}")
            matches = x.objects.exclude(fc_match=False)
            for item in matches.exclude(**filter_expr):
                arche_id = get_p4d_id(item)
                if arche_id in arche_dict:
                    pass
                else:
                    arche_dict[arche_id] = []
                actor = getattr(item, key)
                try:
                    actor_leg = (value, arche_actor_id(actor.legacy_id))
                    arche_dict[arche_id].append(actor_leg)
                except AttributeError:
                    for a in actor.get_queryset():
                        arche_dict[arche_id].append((value, arche_actor_id(a.legacy_id)))

In [ ]:
import pickle

In [ ]:
file = open('arche_contributors.pickle', 'wb')

In [ ]:
pickle.dump(arche_dict, file)

In [ ]:
file.close()

In [ ]:
from rdflib import Graph, URIRef

In [ ]:
for key, value in arche_dict.items():
    g = Graph()
    sub = URIRef(key)
    for tr in value:
        g.add(
            (sub, acdh_ns[tr[0]], URIRef(tr[1]))
        )
    tr_string = g.serialize(format='turtle').decode("utf-8")
    fc_res = FcResource.objects.get(fc_arche_id=key)
    fc_res.fc_custom_rdf = str(tr_string)
    fc_res.save()